#### This notebook download stock prices from Yahoo Finance 

In [ ]:
# df_adjOHLCV has data only for NYSE trading days, no weekend data
# df_adjOHLCV includes data for weekends when BTC trades
# read symbols in file to list syms_in_file
# download OHLCV data for symbols in syms_in_file
# drop symbols with all NaN in OHLCV columns from df
# rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
# drop weekend data by reindex to date index of index_symbol
# pickled df_adjOHLCV
# pickled df_adjOHLCV
# pickled symbols_df_adjOHLCV
# create df_symbols_close, sort df by symbols
# pickled df_symbols_close

In [1]:
import os
import sys

current_dir = os.getcwd()
# Parent directory where myUtils is located
path_utils = os.path.dirname(current_dir)
sys.path.append(path_utils)

In [2]:
# https://stackoverflow.com/questions/67690778/how-to-detect-failed-downloads-using-yfinance
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj, adjust_OHLC

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

verbose = False  # True prints more output

#####################################
# path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_dir = "G:/My Drive/stocks/yfinance/"

##########################################
path_data_dump = path_dir + "VSCode_dump/"
# path_data_dump = path_dir + "VSCode_dump_trash/"


# filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'vg_symbols_4chars_max_clean.csv'  # symbols text file
filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded'  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV

In [3]:
# # function to adjust OHLCV
# # https://github.com/ranaroussi/yfinance/issues/687
# def auto_adjust(data):
#     df = data.copy()
#     ratio = df["Close"] / df["Adj Close"]
#     df["Adj Open"] = df["Open"] / ratio
#     df["Adj High"] = df["High"] / ratio
#     df["Adj Low"] = df["Low"] / ratio

#     df.drop(
#         ["Open", "High", "Low", "Close"],
#         axis=1, inplace=True)

#     df.rename(columns={
#         "Adj Open": "Open", "Adj High": "High",
#         "Adj Low": "Low", "Adj Close": "Close"
#     }, inplace=True)

#     # df = df[["Open", "High", "Low", "Close", "Volume"]]
#     return df[["Open", "High", "Low", "Close", "Volume"]]

def close_vs_Yahoo (symbols, df):
  for symbol in symbols:
    s = df.iloc[-250]
    
##############################################
    # sDate = s.name.strftime('%Y-%m-%d')
    sDate = s.name
    
    sClose = s[symbol].Close

    df_sym = yf.Ticker(symbol).history(period='2y')
    # 10:45PM got Adj Close
    yhClose = df_sym.loc[sDate]['Close']

    abs_pct_diff = abs(1 - sClose/yhClose)*100
    print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
    Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
    if abs_pct_diff > .0001:
      msg_err = f'{symbol}  %_dif_Close > .0001'
      # raise SystemExit(msg_err)
      print(msg_err)  # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
    if symbol == symbols[-1]:
      msg_done = f"No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close "
      print(msg_done)


In [4]:
# Stop if last date in df_adjOHLCV is the same as Yahoo download
index_symbol = "XOM"  

########################################
# df_XOM = yf.download(index_symbol)
# df_XOM = yf.download(index_symbol, period="10y")
df_XOM = yf_download_AdjOHLCV_noAutoAdj(index_symbol, period="10y")

# Change the DatetimeIndex to "yyyy-mm-dd" format
df_XOM.index = df_XOM.index.strftime('%Y-%m-%d')

# download_last_date = df_XOM.index[-1].strftime('%Y-%m-%d')
download_last_date = df_XOM.index[-1]

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
print(f'download_last_date: {download_last_date}')

df = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

# # Change the DatetimeIndex to "yyyy-mm-dd" format
# df.index = df.index.strftime('%Y-%m-%d')

df_adjOHLCV_last_date = df.index[-1]
print(f'df_adjOHLCV_last_date: {df_adjOHLCV_last_date}')

# #####  SystemExit if data have been downloaded  #####
# if download_last_date == df_adjOHLCV_last_date:
#   msg_stop = f"df_adjOHLCV is up-to-date,\nlast date from df_adjOHLCV is {df_adjOHLCV_last_date},\nlast date from Yahoo's {index_symbol} download is {download_last_date}"
#   raise SystemExit(msg_stop )
# #####################################################

[*********************100%***********************]  1 of 1 completed


Full path to pickled df_adjOHLCV:  G:/My Drive/stocks/yfinance/VSCode_dump/df_adjOHLCV
download_last_date: 2024-09-03
df_adjOHLCV_last_date: 2024-08-30


In [7]:
# verify df test_symbols' close against Yahoo
# even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
test_symbols = ['A', 'LMT', 'SPY']
_df_list=[]
_df = yf_download_AdjOHLCV_noAutoAdj(test_symbols, verbose=False)

# Change the DatetimeIndex to "yyyy-mm-dd" format
_df.index = _df.index.strftime('%Y-%m-%d')

_df_list.append(_df)
_df = pd.concat(_df_list, axis=1)
# get unique symbols from column name
_l_symbols = list(set(_df.columns.get_level_values(0)))
_l_symbols.sort()

_df_adj_list=[]
# adjust OHLC using 'Adj Close'
for symbol in _l_symbols:
  _df1 = adjust_OHLC(_df[symbol])
  _df_adj_list.append(_df1)

_df_adj = pd.concat(_df_adj_list, axis=1)
_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# create multilevel column names
_col_names = pd.MultiIndex.from_product([_l_symbols, _cols])
_df_adj.columns = _col_names

close_vs_Yahoo(test_symbols, _df_adj)

[*********************100%***********************]  3 of 3 completed


symbol:     A   Date: 2023-09-06   df_Close:  117.09605     Yahoo_Close:  117.09605   %_dif_Close: 0.00000
symbol:   LMT   Date: 2023-09-06   df_Close:  415.07458     Yahoo_Close:  415.07458   %_dif_Close: 0.00000
symbol:   SPY   Date: 2023-09-06   df_Close:  440.06461     Yahoo_Close:  440.06461   %_dif_Close: 0.00000
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [8]:
# read symbols in file to a list and convert to chunks
#  each chunk must have more than 1 symbol, otherwise, symbol won't appear as column name
symbols_in_file = read_symbols_file(filename_symbols)
print(f'symbols in file: {len(symbols_in_file)}')
symbols_add = ['SPY', 'VGT', 'FTEC', 'QQQ', 'VWO', 'VTV', 'BNDX', 'USO', 'JJC', 'BCI', 'NG', 'GBTC', 'BTC-USD', 'ETH-USD', 'SGOV', 'GLD', 'MSTR']
print(f'symbols added:   {len(symbols_add)}')
symbols = symbols_in_file + symbols_add
# https://stackoverflow.com/questions/12897374/get-unique-values-from-a-list-in-python
symbols = list(set(symbols))  # keep unique symbols
symbols.sort()  # sort unique symbols in alphabetical order
print(f'unique symbols:  {len(symbols)}')
# symbols_chunks = chunked_list(symbols, 400)  # e.g. [['A', 'BB', ...], ['CC', 'DD', ...], ..., ['Z', 'ZWS', ...]]
symbols_chunks = chunked_list(symbols, 400)  # e.g. [['A', 'BB', ...], ['CC', 'DD', ...], ..., ['Z', 'ZWS', ...]]
print(f'number of chucks: {len(symbols_chunks)}')
for chunk in symbols_chunks:
  if len(chunk) == 1:
    raise SystemExit(f'ERROR, Must be more than 1 symbol in each chunk\nsymbols in chunk: {chunk}')

symbols in file: 1593
symbols added:   17
unique symbols:  1602
number of chucks: 17


In [ ]:
df_list=[]
symbols_download_err = []
# took 24 minutes to download 1917 symbols without error caused by Yahoo
# for i, symbols in enumerate(symbols_chunks):
for i, symbols in enumerate(symbols_chunks):
  df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)

#####################################################  
  # Change the DatetimeIndex to "yyyy-mm-dd" format
  df.index = df.index.strftime('%Y-%m-%d')  

  symbols_download_err.append(list(shared._ERRORS.keys()))
  # print(f'symbols_download_err: {symbols_download_err}')
  print(f'\nsymbols_download_err: {symbols_download_err}')  
  df_list.append(df)
  # pause between download
  if i < len(symbols_chunks) - 1 :  # skip pause after last download
    print(f'downloaded symbols from chuck {i}, sleep start')
    # sleep in sec
    time.sleep(60)
    print(f'downloaded symbols from chuck {i}, sleep ends')
  else:
    print(f'downloaded symbols from all chucks')

print(f'symbols_download_err: {symbols_download_err}')

In [ ]:
# flatten symbols_download_err which is a list-of-lists
l_symbols_err = [val for sublist in symbols_download_err for val in sublist]
pickle_dump(l_symbols_err, path_data_dump, 'l_symbols_err')
l_symbols_err

In [10]:
# if l_symbols_err:  # re-download symbols with download error 
#   symbols_download_err = []
#   # took 24 minutes to download 1917 symbols without error caused by Yahoo
#   # for i, symbols in enumerate(symbols_chunks):
#   for i, symbols in enumerate(l_symbols_err):
    # df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)
    # # Change the DatetimeIndex to "yyyy-mm-dd" format
    # df.index = df.index.strftime('%Y-%m-%d')  
#     symbols_download_err.append(list(shared._ERRORS.keys()))
#     # print(f'symbols_download_err: {symbols_download_err}')
#     print(f'\nsymbols_download_err: {symbols_download_err}')  
#     df_list.append(df)
#     # pause between download
#     if i < len(symbols_chunks) - 1 :  # skip pause after last download
#       print(f'downloaded symbols from chuck {i}, sleep start')
#       # sleep 78(18m 25s)
#       time.sleep(78)
#       print(f'downloaded symbols from chuck {i}, sleep ends')
#     else:
#       print(f'downloaded symbols from all chucks')

#   print(f'symbols_download_err: {symbols_download_err}')

In [13]:
df = pd.concat(df_list, axis=1)
if l_symbols_err:  # if list is not empty, drop symbols with errors
  print(f'l_symbols_err is not empty: {l_symbols_err}')  
  df.drop(l_symbols_err, axis=1, level=0, inplace=True)  # drop symbols with errors

l_symbols = list(set(df.columns.get_level_values(0)))  # get unique symbols
l_symbols.sort()
df = df[l_symbols]  # sort columns by list
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
pickle_dump(df, path_data_dump, filename_pickled_df_OHLCVA_downloaded)  # save OHLCA data

l_symbols_err is not empty: ['AJRD', 'ALGM', 'AAWW', 'ABNB', 'ABC', 'ABB', 'AIRC', 'ATCO', 'ATVI', 'BKI', 'CEG', 'CARR', 'BVH', 'CHK', 'CDAY', 'BRBR', 'DCP', 'CR', 'CNHI', 'CNXC', 'CPE', 'DISH', 'ESAB', 'DOOR', 'DDOG', 'ERF', 'DTM', 'FYBR', 'GEHC', 'FISV', 'FLT', 'EVBG', 'FRC', 'GXO', 'GFS', 'HEP', 'HHC', 'HZNP', 'ICPT', 'KDNY', 'IAA', 'KD', 'ISEE', 'LCID', 'LSI', 'MDC', 'MODN', 'NCR', 'NARI', 'NATI', 'MMP', 'MP', 'MGI', 'NEX', 'OSTK', 'NUVA', 'PACW', 'OGN', 'PDCE', 'PEAK', 'OFC', 'OTIS', 'NVST', 'RE', 'RIVN', 'PKI', 'PGNY', 'PXD', 'SGOV', 'SJR', 'SEAS', 'SGEN', 'SIX', 'ROCC', 'SCHN', 'SBNY', 'SITM', 'SHC', 'SYNH', 'SRC', 'SWAV', 'TA', 'TGH', 'SPLK', 'VMW', 'TWNK', 'UNVR', 'TNP', 'WIRE', 'VNT', 'VSCO', 'WWE']
Full path to pickled df_OHLCVA_downloaded:  G:/My Drive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded


In [15]:
# adjust OHLC
df_adj_list=[]
for symbol in l_symbols:
  _df = adjust_OHLC(df[symbol])
  df_adj_list.append(_df)

df_adj = pd.concat(df_adj_list, axis=1)
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
col_names = pd.MultiIndex.from_product([l_symbols, cols])  # create multilevel column names
df_adj.columns = col_names  # reindex columns

print(f"Full path to pickled df_adjOHLCV: {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adj, path_data_dump, filename_pickled_df_adjOHLCV)  # save adjusted OHLCV data

Full path to pickled df_adjOHLCV: G:/My Drive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [16]:
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV)

In [17]:
# drop symbols with all NaN in OHLCV columns from df
df_adjOHLCV, symbols_OHLCV, symbols_dropped = drop_symbols_all_NaN(df_adjOHLCV, verbose)
print(f'symbols with all NaN dropped from df_adjOHLCV: {symbols_dropped}')

Symbol's OHLCV are all NaN: []
symbols with all NaN dropped from df_adjOHLCV: []


In [18]:
# rename columns OHLCV *ONLY AFTER* dropping symbols with all NaN from df,
#   symbols with all NaN has an added AdjClose column and will cause errors  
#  rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
#  .remove_unused_levels() prevents ValueError
#   e.g ValueError: On level 1, code max (5) >= length of level (5). NOTE: this index is in an inconsistent state
# The error may be caused by removing symbols from the dataframe with all NaN in OHLCV columns
df_adjOHLCV.columns = df_adjOHLCV.columns.remove_unused_levels()
# # set_levels reorders df columns in alphabetical order, so the list of column names also needs to be in alphabetical order
# df_adjOHLCV.columns = df_adjOHLCV.columns.set_levels(['close', 'high', 'low', 'open', 'volume'], level=1)

In [20]:
# drop weekend data by re-indexing to date-index of index_symbol
myNaN = float('nan')
# use Exxon's date as proxy for NYSE trading dates
df_adjOHLCV = df_adjOHLCV.reindex(df_XOM.index, fill_value=myNaN)

In [21]:
###########################
df_adjOHLCV

A                                                        AA                                                    AAL                                                    AAON                                                    AAP                                                       AAPL                                                          AB                                                   ABBV                                                        ABM                                                   ABR                                                     ABT                                                       ACGL                                                      ACHC                                                   ACIW                                                   ACLS                                                       ACM                                                       ACN                                                 ACRS              \
                  Open        High         Low       Close     Volume       Open       High        Low      Close     Volume       Open       High        Low      Close      Volume       Open       High        Low      Close    Volume        Open        High         Low       Close     Volume        Open        High         Low       Close       Volume       Open       High        Low      Close    Volume        Open        High         Low       Close      Volume       Open       High        Low      Close    Volume       Open       High        Low      Close     Volume        Open        High         Low       Close     Volume        Open        High         Low       Close     Volume       Open       High        Low      Close     Volume       Open       High        Low      Close    Volume        Open        High         Low       Close    Volume        Open        High        Low       Close     Volume        Open        High         Low       Close     Volume  Open  High   Low   
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
2014-09-04   38.235591   38.479004   37.893498   37.952705  1105259.0  38.546726  39.409373  38.387818  38.819141  6497961.0  37.209812  37.786342  36.255231  36.359196  12004000.0  11.541108  11.976388  11.541108  11.752530  247650.0  125.137709  126.295888  124.821014  125.110565   439400.0   21.942405   22.217656   21.707110   21.780363  342872000.0  11.670367  11.730457  11.511557  11.537310  202700.0   36.620360   36.779118   36.441756   36.739429  10019400.0  22.981344  23.785691  22.784362  22.948513  476500.0   2.666927   2.674601   2.640066   2.640066    80300.0   35.352046   35.583324   35.244668   35.434647  4123200.0   18.496668   18.553333   18.360001   18.376667  2466900.0  51.419998  51.540001  49.349998  49.470001   261900.0  19.549999  19.610001  19.309999  19.389999  258700.0    8.000000    8.040000    7.800000    7.960000   75500.0   36.511083   36.920993  35.925502   36.110935  1748000.0   68.980082   68.996881   68.459412   68.660965  2781100.0   NaN   NaN   NaN   
2014-09-05   37.8

In [22]:
# pickle df_adjOHLCV and symbols
# print(f"Full path to pickled df_adjOHLCV_downloaded:  {path_data_dump}{filename_pickled_df_adjOHLCV_downloaded}")
# pickle_dump(df_adjOHLCV_downloaded, path_data_dump, filename_pickled_df_adjOHLCV_downloaded, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adjOHLCV, path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
pickle_dump(symbols_OHLCV, path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_adjOHLCV:  G:/My Drive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled symbols_df_adjOHLCV:  G:/My Drive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [23]:
symbols_OHLCV = list(set([i[0] for i in list(df_adjOHLCV)]))
df_symbols_close = df_adjOHLCV.xs('Close', level=1, axis=1)
pickle_dump(df_symbols_close, path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

In [24]:
# retrieve pickled files

print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA_downloaded = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
symbols_df = pickle_load(path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_OHLCVA_downloaded:  G:/My Drive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded
Full path to pickled df_adjOHLCV:  G:/My Drive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled df_symbols_close:  G:/My Drive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled symbols_df_adjOHLCV:  G:/My Drive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [25]:
print(f'df_OHLCVA_downloaded.info():\n{df_OHLCVA_downloaded.info()}\n')
print(f'df_adjOHLCV.info():\n{df_adjOHLCV.info()}\n')
print(f'df_close.info():\n{df_close.info()}\n')
print(f'len(symbols_df):\n{len(symbols_df)}\n')

<class 'pandas.core.frame.DataFrame'>
Index: 3650 entries, 2014-09-04 to 2024-09-04
Columns: 9060 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(9060)
memory usage: 252.3+ MB
df_OHLCVA_downloaded.info():
None

<class 'pandas.core.frame.DataFrame'>
Index: 2516 entries, 2014-09-04 to 2024-09-03
Columns: 7550 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(7550)
memory usage: 144.9+ MB
df_adjOHLCV.info():
None

<class 'pandas.core.frame.DataFrame'>
Index: 2516 entries, 2014-09-04 to 2024-09-03
Columns: 1510 entries, A to ZWS
dtypes: float64(1510)
memory usage: 29.0+ MB
df_close.info():
None

len(symbols_df):
1510



In [26]:
count_symbols = len(symbols_df)
count_df_close_cols = len(df_close.columns)
count_cols_df_OHLCVA_downloaded = len(df_OHLCVA_downloaded.columns)
count_cols_df_adjOHLCV = len(df_adjOHLCV.columns)
if count_symbols != count_df_close_cols:
  raise SystemExit(f'symbol count: {count_symbols} does not equal df_close column count: {count_df_close_cols}')
if count_symbols * 6 != count_cols_df_OHLCVA_downloaded:
  raise SystemExit(f'6 x symbol count: {6 * count_symbols} does not equal df_OHLCVA_downloaded column count: {count_cols_df_OHLCVA_downloaded}')
if count_symbols * 5 != count_cols_df_adjOHLCV:
  raise SystemExit(f'5 x symbol count: {5 * count_symbols} does not equal df_adjOHLCV column count: {count_cols_df_adjOHLCV}')
print(f'No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count')


No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count


In [27]:
print(f'count_symbols: {count_symbols}')
print(f'count_df_close_cols: {count_df_close_cols}')
print(f'count_cols_df_OHLCVA_downloaded: {count_cols_df_OHLCVA_downloaded}')
print(f'count_cols_df_adjOHLCV: {count_cols_df_adjOHLCV}')

count_symbols: 1510
count_df_close_cols: 1510
count_cols_df_OHLCVA_downloaded: 9060
count_cols_df_adjOHLCV: 7550


In [28]:
close_vs_Yahoo(test_symbols, df_adjOHLCV)

symbol:     A   Date: 2023-09-06   df_Close:  117.09605     Yahoo_Close:  117.09605   %_dif_Close: 0.00000
symbol:   LMT   Date: 2023-09-06   df_Close:  415.07458     Yahoo_Close:  415.07458   %_dif_Close: 0.00000
symbol:   SPY   Date: 2023-09-06   df_Close:  440.06464     Yahoo_Close:  440.06464   %_dif_Close: 0.00000
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [31]:
syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
_df = df_close[syms_def]
_df.tail()


,NOC,HII,AVAV,LMT,CW,HEI
Date,,,,,,
2024-08-27,512.440002,275.637329,177.759995,560.409973,315.739990,247.389999
2024-08-28,513.559998,278.762909,193.860001,563.979980,312.160004,250.350006
2024-08-29,519.349976,281.559967,203.610001,566.849976,314.399994,255.020004
2024-08-30,523.210022,282.769989,203.759995,568.099976,315.859985,256.559998
2024-09-03,522.690002,275.450012,192.259995,567.219971,302.970001,247.740005
